<a href="https://colab.research.google.com/github/mgondeck/wind_curtailment_prediction/blob/mary/dl_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# packages
import os
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

from sklearn.metrics import classification_report

from matplotlib import pyplot as plt

In [17]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
df = pd.read_csv('/content/drive/My Drive/ms_wind_curtailment_prediction/curtailment_target_features.csv', sep = ';', index_col=0)

In [19]:
# Split the entire dataset into training and test sets
cutoff_time = "2023-01-01"
train = df[df.index < cutoff_time]
test = df[df.index >= cutoff_time]
X_train = train.drop('redispatch', axis = 1)
y_train = train['redispatch']
X_test = test.drop('redispatch', axis = 1)
y_test = test['redispatch']

In [20]:
# preprocessing pipelines
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

smote = SMOTE(random_state=13)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_train_preprocessed, y_train_preprocessed = smote.fit_resample(X_train_preprocessed, y_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [37]:
# define number of epochs and learning rate decay
N_TRAIN = len(X_train_preprocessed)
EPOCHS = 5
BATCH_SIZE = 50
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.01,
    decay_steps=STEPS_PER_EPOCH,
    decay_rate=1,
    staircase=False)

# define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, name='Adam')

# define model
def dl_bin_class_model():
    model = tf.keras.Sequential([
        Dense(64, activation='relu', input_dim=12),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')])

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Recall()])
    return model

# train model
model = dl_bin_class_model()
with tf.device('/device:GPU:0'):
  history = model.fit(X_train_preprocessed,
                    y_train_preprocessed,
                    validation_split=0.2,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1)

Epoch 1/5
3179/3179 [==============================] - 9s 3ms/step - loss: 0.3773 - accuracy: 0.8236 - recall_5: 0.7646 - val_loss: 0.3273 - val_accuracy: 0.8829 - val_recall_5: 0.8829
Epoch 2/5
3179/3179 [==============================] - 8s 2ms/step - loss: 0.2600 - accuracy: 0.8910 - recall_5: 0.8747 - val_loss: 0.2562 - val_accuracy: 0.9155 - val_recall_5: 0.9155
Epoch 3/5
3179/3179 [==============================] - 8s 2ms/step - loss: 0.2138 - accuracy: 0.9143 - recall_5: 0.9070 - val_loss: 0.2051 - val_accuracy: 0.9356 - val_recall_5: 0.9356
Epoch 4/5
3179/3179 [==============================] - 8s 2ms/step - loss: 0.1877 - accuracy: 0.9274 - recall_5: 0.9255 - val_loss: 0.2259 - val_accuracy: 0.9282 - val_recall_5: 0.9282
Epoch 5/5
3179/3179 [==============================] - 8s 2ms/step - loss: 0.1713 - accuracy: 0.9340 - recall_5: 0.9337 - val_loss: 0.2161 - val_accuracy: 0.9297 - val_recall_5: 0.9297


In [38]:
y_pred = model.predict(X_test_preprocessed)

threshold = 0.3 # counteract class imbalaance?!
y_pred_binary = (y_pred >= threshold).astype(int)

# Print classification report
print(classification_report(y_test, y_pred_binary))

1093/1093 [==============================] - 2s 1ms/step
              precision    recall  f1-score   support

         0.0       0.93      0.83      0.88     32472
         1.0       0.09      0.21      0.12      2484

    accuracy                           0.79     34956
   macro avg       0.51      0.52      0.50     34956
weighted avg       0.87      0.79      0.83     34956



In [40]:
# plot recall
plt.plot(history.history['recall'], label='train_recall')
plt.plot(history.history['val_recall'], label='val_recall')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()
plt.title('Training and Validation Recall')

# plot loss
plt.figure()
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')




TypeError: 'History' object is not subscriptable

In [ ]:
# is it overfitting?
# Batchnormalization: method to make artificial NNs faster and more stable through re-centering and re-scaling layers inputs
# Dropout: regularisation technique for reducing overfitting in ANNs by preventing complex co-adaptations on training data. Performing model averaging


In [11]:
'''
## safe
!mkdir -p saved_model
large_model.save('saved_model/my_large_model')

!zip -r saved_model.zip saved_model

# push to GitHub see [3]

## load
# unzip files
import zipfile
with zipfile.ZipFile(‘./saved_model.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

# Load the saved model
with tf.device('/device:GPU:0'):
    new_large_model = tf.keras.models.load_model('saved_model/my_large_model')

# Check its architecture
new_large_model.summary()'''

"\n## safe \n!mkdir -p saved_model\nlarge_model.save('saved_model/my_large_model')\n\n!zip -r saved_model.zip saved_model\n\n# push to GitHub see [3]\n\n## load\n# unzip files \nimport zipfile\nwith zipfile.ZipFile(‘./saved_model.zip', 'r') as zip_ref:\n    zip_ref.extractall('./')\n\n# Load the saved model\nwith tf.device('/device:GPU:0'):\n    new_large_model = tf.keras.models.load_model('saved_model/my_large_model')\n\n# Check its architecture\nnew_large_model.summary()"